# Data Analysis in Python Using Pandas
### by Ethan Orlowsky
#### Credits to Pandas.org and Quantopian.com for the content in this Workshop

## Goals:
This workbook takes you through a guided example of the steps you should take when analyzing datasets for your projects. The outcomes of this workshop are:
- Making calculations on your dataset
- Visualizing data and finding trends in your dataset

## Research Question: How accurate are NFL predraft rankings of prospects?
<img src='images\grading_scale.png' style='width:1500px'><img src='images\baker.png' style='width:1500px'>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

I found my data on [NFL.com] and [https://www.pro-football-reference.com/]. The data isn't very easy to get and requires webscraping. For easy to use data, look at [Kaggle.com] for free datasets you can download.

In [ ]:
player_grades = pd.read_csv('data/player_grades.csv')
player_stats = pd.read_csv('data/player_stats.csv')

In [ ]:
player_grades.head(5)

In [ ]:
player_stats.head(5)

## Pandas Data Structures
### Series
A series is essentially a list of data. A series has two parts: the index and the values
- the index identifies a row
- the values are the data you're actually interested in

In [ ]:
series = player_stats.loc[:, 'name']
print(type(series), '\n')
series.head(5)

#### Accessing Data in Series
We can access data in a series by using the `iloc[]` method
- `iloc[]` accesses elements by the integer index

In [ ]:
# Select the entry at the index 1018
series.iloc[1018]

### DataFrames
A DataFrame is a collection of Series's. One row of a DataFrame is all the data associated with the same index

In [ ]:
print(type(player_stats))
player_stats.head(5)

#### Accessing Data in DataFrames
For DataFrames, we use `loc[]` to access data (note: Series used `iloc[]`)

In [ ]:
# Select the row at the index 1018
player_stats.loc[1018]

In [ ]:
# All the names in the dataframe
# ':' selects all the rows
player_stats.loc[:, 'name']

In [ ]:
# Khalil Mack's awards
player_stats.loc[4, 'gs2']

### Try to print out only the awards Baker Mayfield has won so far

In [ ]:
# Use the above cells for reference. Hint: Baker Mayfield has not won any awards
player_stats.loc[1018, 'awards']

## Making Calculations on DataFrames
You can filter a DataFrame with the `loc[]` method. For our example, we want to filter out players who have an invalid grade.

In [ ]:
# Check which rows have a grade less than 5
player_grades.loc[:, 'grade'] < 5

In [ ]:
# A DataFrame containing the rows where the grade is less than 5
player_grades.loc[player_grades.loc[:, 'grade'] < 5].head(5)

In [ ]:
# Filter our data set to have rows that have grades at least 5
player_grades = player_grades[player_grades.loc[:, 'grade'] >= 5]

In [ ]:
# Check again to see if we still have rows with a grade of less than 5
player_grades.loc[player_grades.loc[:, 'grade'] < 5].head(5)

### Try to find someone on your favorite team
Is their score higher or lower than you expected? Remember this score for later on in the workshop.

In [ ]:
# Replace 'Baker Mayfield' with a different name
# Note: This data only has people drafted between 2014 and 2018, so not all players can be found
player_grades[player_grades.loc[:, 'name'] == 'Baker Mayfield']

### Filling in missing data
For some of the columns, no data can be found. Pandas defaults to calling these missing values `NaN` which stands for "Not a Number." It's hard to perform calculations on a dataset when not all of the values are numbers, so it is neccesary that we change the `NaN` values to a number, or delete entries containg `NaN`

In [ ]:
# Filter the DataFrame to show only the rows that have an instance of a NaN
player_stats.loc[player_stats.loc[:,'g1'].isna()].head(1)

Pandas has a convenient function called `fillna` that replaces all instances of `NaN` with a specified value. In this case, it makes sense to replace `NaN` with 0, but this isn't always the case.

In [ ]:
# Replace all the instances of NaN with the number 0
player_stats = player_stats.fillna(0)

In [ ]:
# Check to see that no rows contain NaN after replacing them with 0
player_stats.loc[player_stats.loc[:,'g1'].isna()].head(1)

### Creating new Columns in DataFrames
The columns in your DataFrame aren't always exactly what you want. For example, in the awards column, we have a list of awards, which we will need to parse through eventually. It will be a lot easier if we create new columns for each of the possible awards that is either True or False.

In [ ]:
# create indicator columns that show whether a certain player has any of these awards
is_mvp = pd.Series(['MVP' in awards for awards in player_stats.loc[:, 'awards']])
is_probowl = pd.Series(['Pro Bowl' in awards for awards in player_stats.loc[:, 'awards']])
is_PoY = pd.Series(['PoY' in awards for awards in player_stats.loc[:, 'awards']])
is_allPro = pd.Series(['All Pro' in awards for awards in player_stats.loc[:, 'awards']])

In [ ]:
is_mvp

In [ ]:
# filter the DataFrame to only show players who have been an MVP
player_stats.loc[is_mvp].head(1)

In [ ]:
# filter the DataFrame to only show players who have made it to the Pro Bowl
player_stats.loc[is_probowl].head(1)

In [ ]:
# filter the DataFrame to only show players who have been a Player of the Year
player_stats.loc[is_PoY].head(1)

In [ ]:
# filter the DataFrame to only show players who have been an All Pro
player_stats.loc[is_allPro].head(1)

In [ ]:
# The following cell creates 4 new indicator columns
player_stats['MVP'] = is_mvp
player_stats['ProBowl'] = is_probowl
player_stats['PoY'] = is_PoY
player_stats['AllPro'] = is_allPro

In [ ]:
# Checking to see that the three new columns were created
player_stats.head(1)

### Removing Columns
Now that we have three awards indicator columns, we don't need the awards columns anymore

In [ ]:
# the first argument of drop is a list of the labels you want to drop
# the second argument of drop says whether you want to drop these labels from the rows (1) or the columns (2)
player_stats = player_stats.drop(['awards'], axis=1)

In [ ]:
# Checking to see that the awards columns was removed
player_stats.head(1)

### Creating a calculated column from multiple columns in a DataFrame
Now that all of the columns in our DataFrame are the way we want, we can now try to calculate a new grade for each player. Using the criteria from NFL.com we can regrade the players.
<img src='images\grading_scale.png' style='width:1500px'> <br>
Here is how I interpreted the grading criteria. You will notice not all grades are possible with the new system, since some of the descriptions are hard to quantify.
- 8 = has been an MVP or PoY
- 7.4 = has been an All Pro
- 7.05 = has been on a Pro Bowl team
- 6.75 = 66% starts in first year with at least 8 starts
- 6.35 = 50% starts in first two years
- 6.15 = 20+ games played in first 2 years, 1+ starts
- 5.85 = 20+ games played in first 2 years, 0 starts
- 5.55 = > 0 games played
- 5.4 = 0 games played

In [ ]:
# Calculating post-draft grades for all the players
grades = pd.Series()
for i in player_stats.index:
    player = player_stats.iloc[i]
    grade = 8 if player['MVP'] or player['PoY'] else \
            7.4 if player['AllPro'] else \
            7.05 if player['ProBowl'] else \
            6.75 if (player['g1'] > 0) and (player['gs1'] / player['g1']) >= 0.66 and player['g1'] >= 8 else \
            6.35 if (player['g1'] + player['g2'] > 0) and (player['gs1'] + player['gs2']) / (player['g1'] + player['g2']) >= 0.5 else \
            6.15 if (player['g1'] + player['g2']) >= 20 and (player['gs1'] + player['gs2']) > 0 else \
            5.85 if (player['g1'] + player['g2']) >= 20 else \
            5.55 if (player['g1'] + player['g2']) > 0 else \
            5.4
    player_series = pd.Series([grade], index=[i])
    grades = grades.append(player_series)

In [ ]:
# adding the post-draft grades as a column into our DataFrame
player_stats['grade'] = grades

In [ ]:
# Checking to see that the new grade column is in the DataFrame
player_stats.head(1)

## Joining two DataFrames together
Since we have two distinct DataFrames, we will need to combine them together to compare the pre-draft and post-draft grades

In [ ]:
# Merging the DataFrame so that rows with the same player name and draft year get combined
merged_df = player_grades.merge(player_stats, on=['name', 'year'], suffixes=['_pre', '_post'])
# Take a subset of the columns that we want
merged_df = merged_df[['name', 'year', 'college', 'position', 'grade_pre', 'grade_post']]
# Calculating the difference between the actual and expected grades
grade_diff = merged_df['grade_post'] - merged_df['grade_pre']
merged_df['grade_diff'] = grade_diff
# Checking to see that the merged DataFrame looks right
merged_df.head(1)

### Check to see your favorite player's pre and post draft grades

In [ ]:
# Replace 'Baker Mayfield' with your favorite player
merged_df[merged_df.loc[:, 'name'] == 'Baker Mayfield']

## Visualizing a Dataset
Now that we have our final dataframe, we want to see how the draft grades compare. We will use some built-in functions to help us see the trends.

In [ ]:
# Plotting the frequencies of draft grades
merged_df[['grade_pre', 'grade_post']].plot.hist(bins=8, figsize=(15,7.5), alpha=0.5, color=['red', 'royalblue'])

plt.xlabel('Grade')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Finding the players who's post draft grades increased by over 1.5
risers = merged_df.loc[merged_df.loc[:, 'grade_diff'] > 1.5]
x = risers['grade_post']
y = risers['grade_diff']

fig, ax = plt.subplots(figsize=(15,7.5))
ax.scatter(x, y)

for i, name in enumerate(risers.loc[:, 'name']):
    ax.annotate(s=name, xy=(x.iloc[i], y.iloc[i]), xytext=(x.iloc[i] - .06, y.iloc[i] + .01))

plt.xlabel('Post-Draft Grade')
plt.ylabel('Grade Difference')
plt.show()

In [ ]:
average_grades = merged_df.groupby('grade_pre').mean().drop(['year','grade_diff'], axis=1)
average_grades['grade_pre'] = average_grades.index
average_grades.plot(figsize=(15,7.5))

plt.xlabel('Average Pre-Draft Grade')
plt.ylabel('Average Post-Draft Grade')
plt.show()